### Neo4j Connection

In [21]:
# import the neo4j driver for Python
from neo4j import GraphDatabase


# Database Credentials
uri             = "bolt://localhost:7687"
userName        = "neo4j"
password        = "R0b3n$0n"
 
# Connect to the neo4j database server
graphDB_conn  = GraphDatabase.driver(uri, auth=(userName, password))


### General Import

In [6]:
import pandas as pd
import pickle
import os
import import_ipynb

%run ./helper_functions/shared_functions_basic.ipynb

### Import the profile of the customers

In [72]:
DIR_INPUT='./Baseline/simulation_data/customer_profile/customer_profile.pkl'

customer_df= pd.read_pickle(DIR_INPUT)

customer_df

,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,available_terminals,nb_terminals
0,0,54.881350,71.518937,62.262521,31.131260,2.179533,"[29, 87, 144, 241, 330, 858, 996]",7
1,1,42.365480,64.589411,46.570785,23.285393,3.567092,"[5, 160, 242, 378, 431, 475, 571, 762, 876, 93...",12
2,2,96.366276,38.344152,80.213879,40.106939,2.115580,"[316, 406, 447, 523, 968]",5
3,3,56.804456,92.559664,11.748426,5.874213,0.348517,"[65, 94, 113, 364, 401, 433, 485, 651, 672, 77...",14
4,4,2.021840,83.261985,78.924891,39.462446,3.480049,"[372, 614, 774]",3
...,...,...,...,...,...,...,...,...
495,495,94.174214,72.104341,33.307175,16.653588,2.836935,"[56, 568, 682, 783, 794]",5
496,496,73.193028,34.222633,40.680913,20.340457,1.436426,"[17, 41, 207, 273, 441, 695, 703, 828]",8
497,497,61.661844,90.041015,21.453357,10.726679,3.500798,"[99, 113, 401, 433, 539, 672, 680]",7
498,498,2.765316,66.033860,44.371693,22.185846,3.165126,"[7, 25, 49, 201, 504]",5


500 customers profile have been created.

### Creation of the nodes Customer

In [73]:
with graphDB_conn.session() as graphDB:
    for index, row in customer_df.iterrows():
        graphDB.run('''MERGE (c:Customer {customerId:$id,latitude:$Lat,longitude:$Long,meanAmount:$MeanAmount, stdAmount:$STDAmount, meanNb_tx_per_day:$MeanTxDay})''',\
                    parameters = {'id': int(row['CUSTOMER_ID']), \
                          'Lat': float(row['x_customer_id']), \
                          'Long': float(row['y_customer_id']), \
                          'MeanAmount': float(row['mean_amount']),\
                          'STDAmount': float(row['std_amount']),\
                          'MeanTxDay': float(row['mean_nb_tx_per_day']) })

### Import the terminals information

In [74]:
DIR='./Baseline/simulation_data/terminal/terminal.pkl'

terminal_df= pd.read_pickle(DIR)

terminal_df

,TERMINAL_ID,x_terminal_id,y_terminal_id
0,0,41.702200,72.032449
1,1,0.011437,30.233257
2,2,14.675589,9.233859
3,3,18.626021,34.556073
4,4,39.676747,53.881673
...,...,...,...
995,995,86.414409,44.789892
996,996,56.178626,73.671096
997,997,79.648887,44.750814
998,998,18.412756,82.873285


1000 terminals have been created

### Creation of the nodes Terminal

In [75]:
with graphDB_conn.session() as graphDB:
    for index, row in terminal_df.iterrows():
        graphDB.run('''MERGE (a:Terminal {terminalId:$id,latitude:$Lat,longitude:$Long})''', \
                    parameters = {'id': int(row['TERMINAL_ID']), \
                          'Lat': float(row['x_terminal_id']), \
                          'Long': float(row['y_terminal_id'])})


### Creation of the relationship "CONNECTED" between Customer and terminal in a radius = 5

In [76]:
with graphDB_conn.session() as graphDB:
    for index, row in customer_df.iterrows():
        graphDB.run('''MATCH (t:Terminal),(c:Customer) WHERE t.terminalId in $Idterm AND c.customerId=$idcust MERGE (c)-[:CONNECTED]->(t)''', \
                    parameters = {'idcust': int(row['CUSTOMER_ID']), \
                          'Idterm': [ int(idterm) for idterm in row['available_terminals']]})

### Import the transaction of 1 day.

In [77]:
DIR_INPUT='./Baseline/simulation_data/transformed_features/' 

BEGIN_DATE = "2018-04-01"
END_DATE = "2018-04-01"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)

Load  files
Wall time: 599 ms


In [71]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0,1,...,1.0,57.160000,1.0,57.160000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0,1,...,1.0,81.510000,1.0,81.510000,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0,1,...,1.0,146.000000,1.0,146.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0,1,...,1.0,64.490000,1.0,64.490000,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0,1,...,1.0,50.990000,1.0,50.990000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9483,9483,2018-04-01 23:56:50,3289,6699,48.69,86210,0,0,0,1,...,4.0,85.175000,4.0,85.175000,0.0,0.0,0.0,0.0,0.0,0.0
9484,9484,2018-04-01 23:58:14,3230,6664,85.97,86294,0,0,0,1,...,7.0,75.205714,7.0,75.205714,0.0,0.0,0.0,0.0,0.0,0.0
9485,9485,2018-04-01 23:58:31,296,3702,120.88,86311,0,0,0,1,...,3.0,111.660000,3.0,111.660000,0.0,0.0,0.0,0.0,0.0,0.0
9486,9486,2018-04-01 23:59:28,2557,146,8.02,86368,0,0,0,1,...,5.0,9.350000,5.0,9.350000,0.0,0.0,0.0,0.0,0.0,0.0


### Creation of the relationship "BROUGHT_AT" between the customer and the terminals

In [80]:
with graphDB_conn.session() as graphDB:
    for index, row in transactions_df.iterrows():
        graphDB.run('''MATCH (t:Terminal),(c:Customer) WHERE t.terminalId=$Idterm AND c.customerId=$idcust 
        MERGE (c)-[:BROUGHT_AT{tx_Amount:$txAmount,tx_During_Weekend:$txDuringWeekend, tx_During_Night:$txDuringNight, customer_Id_Nb_Tx_1Day_Window:$customerIdNbTx1DayWindow,
       customer_Id_Avg_Amount_1day_Window:$customerIdAvgAmount1dayWindow, customer_Id_Nb_Tx_7Day_Window:$customerIdNbTx7DayWindow,
       customer_Id_Avg_Amount_7Day_Window:$customerIdAvgAmount7DayWindow, customer_Id_Nb_Tx_30Day_Window:$customerIdNbTx30DayWindow,
       customer_Id_Avg_Amount_30Day_Window:$customerIdAvgAmount30DayWindow, terminal_Id_Nb_Tx_1Day_Window:$terminalIdNbTx1DayWindow,
       terminal_Id_Risk_1Day_Window:$terminalIdRisk1DayWindow , terminal_Id_Nb_Tx_7Day_Window:$terminalIdNbTx7DayWindow,
       terminal_Id_Risk_7Day_Window:$terminalIdRisk7DayWindow, terminal_Id_Nb_Tx_30Day_Window:$terminalIdNbTx30DayWindow,
       terminal_Id_Risk_30Day_Window:$terminalIdRisk30DayWindow,tx_Fraud:$txFraud}]->(t)''', \
                    parameters = {'idcust': int(row['CUSTOMER_ID']),\
                                  'Idterm': int(row['TERMINAL_ID']),\
                                  'txAmount':float(row['TX_AMOUNT']),\
                                  'txDuringWeekend':float(row['TX_DURING_WEEKEND']),\
                                  'txDuringNight': float(row['TX_DURING_NIGHT']),\
                                  'customerIdNbTx1DayWindow': int(row['CUSTOMER_ID_NB_TX_1DAY_WINDOW']),\
                                  'customerIdAvgAmount1dayWindow':float(row['CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW']),\
                                  'customerIdNbTx7DayWindow':int(row['CUSTOMER_ID_NB_TX_7DAY_WINDOW']),\
                                  'customerIdAvgAmount7DayWindow': float(row['CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW']),\
                                  'customerIdNbTx30DayWindow': int(row['CUSTOMER_ID_NB_TX_30DAY_WINDOW']),\
                                  'customerIdAvgAmount30DayWindow':float(row['CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW']),\
                                  'terminalIdNbTx1DayWindow':int(row['TERMINAL_ID_NB_TX_1DAY_WINDOW']),\
                                  'terminalIdRisk1DayWindow': float(row['TERMINAL_ID_RISK_1DAY_WINDOW']),\
                                  'terminalIdNbTx7DayWindow': int(row['TERMINAL_ID_NB_TX_7DAY_WINDOW']),\
                                  'terminalIdRisk7DayWindow':float(row['TERMINAL_ID_RISK_7DAY_WINDOW']),\
                                  'terminalIdNbTx30DayWindow':int(row['TERMINAL_ID_NB_TX_30DAY_WINDOW']),\
                                  'terminalIdRisk30DayWindow': float(row['TERMINAL_ID_RISK_30DAY_WINDOW']),\
                                  'txFraud': int(row['TX_FRAUD'])
                                 })